# Create Input Data

This script takes a variety of chemical identifiers for both insecticidal and non-insecticidal compounds, cleans these chemicals, converts them to SMILES, and then converts SMILES to graphs.

Written by Tobias D. Muellers

## Load Dependencies

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
from selenium import webdriver
import cirpy
from tqdm import tqdm
import urllib.request
from urllib.error import HTTPError
from urllib.request import urlopen
import rdkit

# Scrape for Chemicals

In [2]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [3]:
# function to get content from https://www.pesticideinfo.org/
# based on https://realpython.com/beautiful-soup-web-scraper-python/
def pesticideinfo_get(PRI_start, PRI_end):
    """
    this function takes starting and ending ids for this database and constructs a range.
    Based on the range of PRIs, it extracts data from each PesticideInfo page. 
    PRI_start = integer
    PRI_end = integer
    """
    # webdriver workaround from https://stackoverflow.com/questions/76928765/attributeerror-str-object-has-no-attribute-capabilities-in-selenium
    cservice = webdriver.ChromeService(
        executable_path="C:/Users/tobia/OneDrive/Documents/GitHub/BugBuster/chromedriver-win64/chromedriver-win64/chromedriver.exe",
        chrome_options=options)
    driver = webdriver.Chrome(service = cservice)
    # driver code based on https://stackoverflow.com/questions/52687372/beautifulsoup-not-returning-complete-html-of-the-page
    
    # make range
    PRIs = np.arange(PRI_start, PRI_end, 1)

    # set up storage for a dataframe
    pris = []
    names = []
    casrns = []
    classes = []
    mws = []
    uses =[]

    for pri in PRIs:
        URL = "https://www.pesticideinfo.org/chemical/PRI"+str(pri)
        driver.get(URL)
        time.sleep(1) # wait for page to load
        page = driver.page_source
        soup = bs(page, "html.parser") # parse page
        table = soup.find_all("div", {"class": "data-table-key-value"}) #get values from table of interest
        
        if len(table) < 5:
            print(f'{pri} does not exist')
        else:
            # now extract desired information 
            pris.append(pri)
            name = str(table[0]).split('</div>')[1][5:]
            names.append(name)
            casrn = str(table[2]).split('</div>')[1][5:]
            casrns.append(casrn)
            chem_class = str(table[4]).split('</div>')[1][5:]
            classes.append(chem_class)
            mw = str(table[5]).split('</div>')[1][5:]
            mws.append(mw)
            use = str(table[6]).split('</div>')[1][5:]
            uses.append(use)

    data = {'name': names, 'pri': pris, 'casrn': casrns, 
            'class': classes, 'mw': mws, 'use': uses}
    df = pd.DataFrame(data)
    
    driver.quit()
    
    return df

In [5]:
#extracted_1_10001 = pesticideinfo_get(1, 10001)
#extracted_10001_20001 = pesticideinfo_get(10001, 20001) this does not seem to pick up much useful information
#extracted_1_10001.tail(10)
#extracted_10001_20001.head(1)
#pest_info_db_all = pd.concat([extracted_1_1001, extracted_1001_2001], axis=0)
#pest_info_db_all = extracted_1_10001.copy()
#pest_info_db_all.to_csv("pest_info_db_1_10001.csv")

# Get SMILES

In [5]:
# load extracted data
raw_pestinfo = pd.read_csv('pest_info_db_1_10001.csv')
raw_pestinfo.head(1)

,Unnamed: 0,name,pri,casrn,class,mw,use
0,0,1-(3-chlorophthalimido)cyclohexanecarboxamide,10,51971-67-6,Carboxamide,0.0,Not Listed


In [133]:
# add smiles based on casrn
# use cirpy 
# add cactus as needed
def add_smiles(df, cas_col, name_col):
    temp_df = df.copy() # avoid overwrite
    casrns = temp_df[cas_col]
    names = temp_df[name_col]
    smiles_cirpy_cas_storage = []
    smiles_cirpy_name_storage = []

    for cas in tqdm(casrns):
        if cas == "Not Listed":
            smiles_cirpy_casrn = float('NaN')
        else:
            smiles_cirpy_casrn = cirpy.resolve(cas, 'smiles')
        smiles_cirpy_cas_storage.append(smiles_cirpy_casrn)

    for name in tqdm(names):
        smiles_cirpy_name = cirpy.resolve(name, 'smiles')
        smiles_cirpy_name_storage.append(smiles_cirpy_name)

    temp_df['smiles_cirpy_casrn'] = smiles_cirpy_cas_storage
    temp_df['smiles_cirpy_name'] = smiles_cirpy_name_storage

    return temp_df

In [135]:
#test = add_smiles(raw_pestinfo.head(10), 'casrn', 'name')
#test

In [136]:
smiles_pestinfo = add_smiles(raw_pestinfo, 'casrn', 'name')

100%|██████████| 6261/6261 [57:49<00:00,  1.80it/s]  


In [138]:
# save intermediate output
smiles_pestinfo.to_csv("pest_info_db_smiles.csv")

## Trim SMILES

In [2]:
# load smiles
smiles_pestinfo = pd.read_csv('pest_info_db_smiles.csv')
#smiles_pestinfo.head(10)

In [3]:
# remove no smiles
def merge_and_remove_smiles(df, cirpy_casrn_smiles, cirpy_name_smiles):
    temp = df.copy()
    idx = range(0, temp.shape[0])
    cirpy_casrn_smiles = temp[cirpy_casrn_smiles]
    cirpy_name_smiles = temp[cirpy_name_smiles]

    smiles_storage = []
    for i in idx:
        if pd.isnull(cirpy_casrn_smiles[i]) == False:
            smiles = cirpy_casrn_smiles[i]
        else:
            if pd.isnull(cirpy_name_smiles[i]) == False:
                smiles = cirpy_name_smiles[i]
            else:
                smiles = float('NaN')
        smiles_storage.append(smiles)

    temp['smiles'] = smiles_storage
    
    return temp

In [4]:
smiles_pestinfo_cleaned = merge_and_remove_smiles(smiles_pestinfo, "smiles_cirpy_casrn", "smiles_cirpy_name")

In [5]:
smiles_pestinfo_cleaned = smiles_pestinfo_cleaned.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'smiles_cirpy_casrn', 'smiles_cirpy_name'])

In [6]:
smiles_pestinfo_cleaned = smiles_pestinfo_cleaned[pd.isnull(smiles_pestinfo_cleaned['smiles']) == False]

In [59]:
#smiles_pestinfo_cleaned.head(5)

# Filters

## Filter for neutral organics

In [7]:
# drop inorganic
print(smiles_pestinfo_cleaned.shape)
organics = smiles_pestinfo_cleaned[smiles_pestinfo_cleaned['class'] != 'Inorganic']
print(organics.shape)

(3822, 7)
(3558, 7)


In [8]:
# drop inorganics based on classes
#organics["class"].unique() # see all classes
print(organics.shape)
organics = pd.DataFrame(organics[~organics['class'].str.contains('Inorganic')]) # this removes where class is inorganic
print(organics.shape)

(3558, 7)
(3388, 7)


In [9]:
from rdkit import Chem
from rdkit.Chem import SaltRemover

# remove salts, https://www.rdkit.org/docs/source/rdkit.Chem.SaltRemover.html
def no_salts(df, smiles_col):
    temp = df.copy()
    smiles = temp[smiles_col]
    cleaned = []
    for smi in smiles:

        # for chlorine
        ions = ['Cl-', 'Br-', 'NH4+', 'Na+', 'Ca+']
        if any(x in smi for x in ions): # https://stackoverflow.com/questions/3389574/check-if-multiple-strings-exist-in-another-string
            remover = SaltRemover.SaltRemover(defnData="[Cl-]", defnFormat='smiles')
            res = remover.StripMol(Chem.MolFromSmiles(smi), dontRemoveEverything=True)
            remover = SaltRemover.SaltRemover(defnData="[Br-]", defnFormat='smiles')
            res = remover.StripMol(res, dontRemoveEverything=True)
            remover = SaltRemover.SaltRemover(defnData="[NH4+]", defnFormat='smiles')
            res = remover.StripMol(res, dontRemoveEverything=True)
            remover = SaltRemover.SaltRemover(defnData="[Na+]", defnFormat='smiles')
            res = remover.StripMol(res, dontRemoveEverything=True)
            remover = SaltRemover.SaltRemover(defnData="[Ca+]", defnFormat='smiles')
            res = Chem.MolToSmiles(remover.StripMol(res, dontRemoveEverything=True))
            cleaned.append(res)
        else:
            cleaned.append(smi)
                
    temp['no_salt'] = cleaned
    return temp

In [10]:
print(organics.shape)
desalted = no_salts(organics, 'smiles')
print(desalted.shape)
#test = organics[organics['smiles'].str.contains('\\[' + 'NH4' + '\\+' + '\\]')]
#test = no_salts(test, 'smiles')
#test.head(10)

(3388, 7)


[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not removing hydrogen atom without neighbors
[16:40:20] WARNING: not r

(3388, 8)


In [12]:
desalted.tail(10)

,name,pri,casrn,class,mw,use,smiles,no_salt
6206,(Z)-9-hexadecenal,6616,56219-04-6,Pheromone,0.00,Pheromone,CCCCCC\C=C/CCCCCCCC=O,CCCCCC\C=C/CCCCCCCC=O
6207,(Z)-9-hexadecenyl acetate,6617,34010-20-3,Pheromone,0.00,Pheromone,CCCCCC\C=C/CCCCCCCCOC(C)=O,CCCCCC\C=C/CCCCCCCCOC(C)=O
6208,(Z)-9-tetradecen-1-ol,6618,35153-15-2,Pheromone,0.00,Pheromone,CCCC\C=C/CCCCCCCCO,CCCC\C=C/CCCCCCCCO
6209,(Z)-9-tetradecenal,6620,53939-27-8,Pheromone,0.00,Pheromone,CCCC\C=C/CCCCCCCC=O,CCCC\C=C/CCCCCCCC=O
6210,(Z)-9-Tricosene,6622,27519-02-4,Pheromone,322.60,Pheromone,CCCCCCCCCCCCC\C=C/CCCCCCCC,CCCCCCCCCCCCC\C=C/CCCCCCCC
6211,(Z)-Mevinphos,6623,338-45-4,Organophosphorus,0.00,Insecticide,COC(=O)/C=C(/C)O[P](=O)(OC)OC,COC(=O)/C=C(/C)O[P](=O)(OC)OC
6212,"(Z)-N,N,N-Trimethyl-9-octadecen-1-aminium chlo...",6624,10450-69-8,Not Listed,0.00,Not Listed,[Cl-].CCCCCCCC\C=C/CCCCCCCC[N+](C)(C)C,CCCCCCCC/C=C\CCCCCCCC[N+](C)(C)C
6255,Zirconyl ammonium carbonate,6669,32535-84-5,Not Listed,0.00,Not Listed,[NH4+].[NH4+].[NH4+].[OH-].[Zr+4].[O-]C([O-])=...,O=C([O-])[O-].O=C([O-])[O-].O=C([O-])[O-].[OH-...
6257,Zoalene,6671,148-01-6,Not Listed,0.00,Not Listed,Cc1c(cc(cc1[N+]([O-])=O)[N+]([O-])=O)C(N)=O,Cc1c(cc(cc1[N+]([O-])=O)[N+]([O-])=O)C(N)=O
6258,Zoxamide,6672,156052-68-5,Not Listed,336.64,Fungicide,CCC(C)(NC(=O)c1cc(Cl)c(C)c(Cl)c1)C(=O)CCl,CCC(C)(NC(=O)c1cc(Cl)c(C)c(Cl)c1)C(=O)CCl


In [ ]:
# neutralize molecules? https://www.rdkit.org/docs/Cookbook.html

In [31]:
# rough filter for now
# remove all not bonded things
import re
print(desalted.shape)
desalted_neut_org = desalted[~desalted['no_salt'].str.contains('\\.')]
print(desalted_neut_org.shape)

(3388, 8)
(2931, 8)


In [35]:
desalted_neut_org.head(5)

,name,pri,casrn,class,mw,use,smiles,no_salt
0,1-(3-chlorophthalimido)cyclohexanecarboxamide,10,51971-67-6,Carboxamide,0.0,Not Listed,NC(=O)C1(CCCCC1)N2C(=O)c3cccc(Cl)c3C2=O,NC(=O)C1(CCCCC1)N2C(=O)c3cccc(Cl)c3C2=O
2,"1-(6-Isopropyl-1,1,4-trimethyl-5-indanyl)-1-pr...",12,6682-77-5,Not Listed,0.0,Not Listed,CCC(=O)c1c(C)c2CCC(C)(C)c2cc1C(C)C,CCC(=O)c1c(C)c2CCC(C)(C)c2cc1C(C)C
4,"1-(8-Methoxy-4,8-dimethylnonyl)-4-(1-methyleth...",14,53905-38-7,Not Listed,0.0,Not Listed,COC(C)(C)CCCC(C)CCCc1ccc(cc1)C(C)C,COC(C)(C)CCCC(C)CCCc1ccc(cc1)C(C)C
14,1-(Bromoacetoxy)-2-propanol,24,4189-47-3,Not Listed,0.0,Not Listed,CC(O)COC(=O)CBr,CC(O)COC(=O)CBr
15,1-(Dodecylbenzyl)pyridinium chloride,25,30901-67-8,Not Listed,0.0,Not Listed,[Cl-].CCCCCCCCCCCCC(c1ccccc1)[n+]2ccccc2,CCCCCCCCCCCCC(c1ccccc1)[n+]1ccccc1


# Use labels

In [78]:
#print(desalted_neut_org["use"].unique())

In [ ]:
all_uses = desalted_neut_org.copy()

In [93]:
all_uses.head(1)

,name,pri,casrn,class,mw,use,smiles,no_salt
0,1-(3-chlorophthalimido)cyclohexanecarboxamide,10,51971-67-6,Carboxamide,0.0,Not Listed,NC(=O)C1(CCCCC1)N2C(=O)c3cccc(Cl)c3C2=O,NC(=O)C1(CCCCC1)N2C(=O)c3cccc(Cl)c3C2=O


In [87]:
# iterative removal, because this fails the later graph creation at the moment
print(all_uses.shape)
Sn = all_uses['no_salt'].str.contains('Sn')
no_failures = all_uses[~Sn]
print(no_failures.shape)
Hg = desalted_neut_org['no_salt'].str.contains('Hg')
no_failures = no_failures[~Hg]
print(no_failures.shape)
Al = desalted_neut_org['no_salt'].str.contains('Al')
no_failures = no_failures[~Al]
print(no_failures.shape)
Bi = desalted_neut_org['no_salt'].str.contains('Bi')
no_failures = no_failures[~Bi]
print(no_failures.shape)
Pb = desalted_neut_org['no_salt'].str.contains('Pb')
no_failures = no_failures[~Pb]
print(no_failures.shape)

(2931, 8)
(2904, 8)
(2870, 8)
(2869, 8)
(2867, 8)
(2866, 8)


C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2775307384.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_failures = no_failures[~Hg]
C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2775307384.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_failures = no_failures[~Al]
C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2775307384.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_failures = no_failures[~Bi]
C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2775307384.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_failures = no_failures[~Pb]


In [88]:
pesticides = no_failures[no_failures['use'].str.contains('|'.join(['Insecticide','Fungicide','Microbiocide',
                                                             'Herbicide','Algaecide','Nematicide','Rodenticide',
                                                            'Molluscicide','Avicide']))]
print(pesticides.shape)
insecticides = pesticides[pesticides['use'].str.contains('Insecticide')]
print(insecticides.shape)
nonactives = 

(1566, 8)
(489, 8)


# Data Augmentation

In [169]:
insecticides.head(10)

,name,pri,casrn,class,mw,use,smiles,no_salt,alogp
35,1-Chloro-2-nitrobenzene,46,88-73-3,Unclassified,157.56,Insecticide,[O-][N+](=O)c1ccccc1Cl,[O-][N+](=O)c1ccccc1Cl,2.24820
38,1-Chloro-3-nitrobenzene,49,121-73-3,Unclassified,157.56,Insecticide,[O-][N+](=O)c1cccc(Cl)c1,[O-][N+](=O)c1cccc(Cl)c1,2.24820
39,1-Chloro-4-nitrobenzene,50,100-00-5,Unclassified,157.56,"Chemical intermediate,Insecticide",[O-][N+](=O)c1ccc(Cl)cc1,[O-][N+](=O)c1ccc(Cl)cc1,2.24820
141,"1,080",162,62-74-8,Unclassified,100.02,"Insecticide,Rodenticide",[Na+].[O-]C(=O)CF,O=C([O-])CF,-1.29420
156,2-(2-butoxy ethoxy) ethyl thiocyanate,180,112-56-1,Not Listed,203.30,Insecticide,CCCCOCCOCCSC#N,CCCCOCCOCCSC#N,2.03398
166,"2-(4,5-Dimethyl-1,3-dioxolan-2-yl)phenyl-N-met...",191,7122-04-5,N-Methyl Carbamate,0.00,Insecticide,CNC(=O)Oc1ccccc1C2OC(C)C(C)O2,CNC(=O)Oc1ccccc1C2OC(C)C(C)O2,2.22720
211,"2-chloro-4,5-xylyl N-hydroxy-N-methylcarbamate",243,"14357-82-5, 10310-17-5",N-Methyl Carbamate,0.00,Insecticide,CN(O)C(=O)Oc1cc(C)c(C)cc1Cl,CN(O)C(=O)Oc1cc(C)c(C)cc1Cl,2.77664
222,"2-cyclohexyl-4,5-dichloro-4-isothioazolin-3-one",255,57063-29-3,Isothiazolone,0.00,Insecticide,ClC1=C(Cl)C(=O)N(S1)C2CCCCC2,ClC1=C(Cl)C(=O)N(S1)C2CCCCC2,3.72180
225,2-cyclopentylphenyl methylcarbamate,258,3282-00-6,N-Methyl Carbamate,0.00,Insecticide,CNC(=O)Oc1ccccc1C2CCCC2,CNC(=O)Oc1ccccc1C2CCCC2,3.06240
250,2-hydroxyethyl octyl sulfide,285,3547-33-9,Not Listed,0.00,Insecticide,CCCCCCCCSCCO,CCCCCCCCSCCO,3.07240


In [253]:
# idea 1
def aliphatic_string(df, col):
    '''
    For each carbon string of 3 or more, create a new molecule with one less carbon and one with one more
    '''
    df_temp = df.copy().reset_index()

    augmented = []
    for index, row in df_temp.iterrows():
        smi = df_temp[col][index]
        if 'CCC' in smi:
            original = df_temp.iloc[index,:] 
            # get the longest CCC.. string
            # this isn't working yet, which means that this adds 3x1 C based on number of CCCs

            plus_C = smi.replace('CCC', 'CCCC')
            plus_C_row = original
            plus_C_row[7] = plus_C
            minus_C = smi.replace('CCC', 'CC')
            minus_C_row = original
            minus_C_row[7] = minus_C
            augmented.append(plus_C_row)
            augmented.append(minus_C_row)
            
    out_df = pd.DataFrame(augmented)
    
    return out_df

In [254]:
aliphatic_string(insecticides, 'no_salt')

C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\4276350094.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plus_C_row[7] = plus_C
C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\4276350094.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minus_C_row[7] = minus_C


,index,name,pri,casrn,class,mw,use,smiles,no_salt,alogp
4,156,2-(2-butoxy ethoxy) ethyl thiocyanate,180,112-56-1,Not Listed,203.30,Insecticide,CCCOCCOCCSC#N,CCCCOCCOCCSC#N,2.03398
4,156,2-(2-butoxy ethoxy) ethyl thiocyanate,180,112-56-1,Not Listed,203.30,Insecticide,CCCOCCOCCSC#N,CCCCOCCOCCSC#N,2.03398
7,222,"2-cyclohexyl-4,5-dichloro-4-isothioazolin-3-one",255,57063-29-3,Isothiazolone,0.00,Insecticide,ClC1=C(Cl)C(=O)N(S1)C2CCCC2,ClC1=C(Cl)C(=O)N(S1)C2CCCCC2,3.72180
7,222,"2-cyclohexyl-4,5-dichloro-4-isothioazolin-3-one",255,57063-29-3,Isothiazolone,0.00,Insecticide,ClC1=C(Cl)C(=O)N(S1)C2CCCC2,ClC1=C(Cl)C(=O)N(S1)C2CCCCC2,3.72180
8,225,2-cyclopentylphenyl methylcarbamate,258,3282-00-6,N-Methyl Carbamate,0.00,Insecticide,CNC(=O)Oc1ccccc1C2CCC2,CNC(=O)Oc1ccccc1C2CCCC2,3.06240
...,...,...,...,...,...,...,...,...,...,...
441,5660,Sulfoxide,6041,120-62-7,Not Listed,324.52,"Insecticide,Synergist",CCCCCC[S](=O)C(C)Cc1ccc2OCOc2c1,CCCCCCCC[S](=O)C(C)Cc1ccc2OCOc2c1,4.45550
442,5671,Sulprofos,6052,35400-43-2,Organophosphorus,322.40,Insecticide,CCS[P](=S)(OCC)Oc1ccc(SC)cc1,CCCS[P](=S)(OCC)Oc1ccc(SC)cc1,5.19150
442,5671,Sulprofos,6052,35400-43-2,Organophosphorus,322.40,Insecticide,CCS[P](=S)(OCC)Oc1ccc(SC)cc1,CCCS[P](=S)(OCC)Oc1ccc(SC)cc1,5.19150
452,5804,Tetramethrin,6195,"7696-12-0, 1166-46-7",Pyrethroid,331.40,Insecticide,CC(C)=C[C@@H]1[C@@H](C(=O)OCN2C(=O)C3=C(CCC3)C...,CC(C)=C[C@@H]1[C@@H](C(=O)OCN2C(=O)C3=C(CCCC3)...,2.96490


# Predict Log P

In [96]:
from rdkit.Chem.QED import properties

In [110]:
def alogp(df, col):
    '''
    calculates logP using QED ALogP
    https://www.rdkit.org/docs/source/rdkit.Chem.QED.html
    ''' 
    logp_storage = []

    smiles = df[col]
    
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        properties = rdkit.Chem.QED.properties(mol)
        logp = properties[1]
        logp_storage.append(logp)

    df['alogp'] = logp_storage

    return df

In [111]:
test_smi = 'NC(=O)C1(CCCCC1)N2C(=O)c3cccc(Cl)c3C2=O'
mol = Chem.MolFromSmiles(test_smi)
logp = rdkit.Chem.QED.properties(mol)
logp[1]

2.1242

In [113]:
insecticides_logp = alogp(insecticides, 'no_salt')
pesticides_logp = alogp(pesticides, 'no_salt')

C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2816114057.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alogp'] = logp_storage
C:\Users\tobia\AppData\Local\Temp\ipykernel_13488\2816114057.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alogp'] = logp_storage


# SMILES to Graphs
from https://www.blopig.com/blog/2022/02/how-to-turn-a-smiles-string-into-a-molecular-graph-for-pytorch-geometric/

In [17]:
# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

### Atom featurization

In [19]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """
    if x not in permitted_list:
        x = permitted_list[-1]
    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]
    return binary_encoding

In [20]:
def get_atom_features(atom, 
                      use_chirality = True, 
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """
    # define list of permitted atoms
    
    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
    
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
    
    # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]
    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
                                    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc
    return np.array(atom_feature_vector)

### Bond featurization

In [21]:
def get_bond_features(bond, 
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """
    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc
    return np.array(bond_feature_vector)

### Function to integrate and create dataset

In [22]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y):
    """
    Inputs:
    
    x_smiles = [smiles_1, smiles_2, ....] ... a list of SMILES strings
    y = [y_1, y_2, ...] ... a list of numerial labels for the SMILES strings (such as associated pKi values)
    
    Outputs:
    
    data_list = [G_1, G_2, ...] ... a list of torch_geometric.data.Data objects which represent labeled molecular graphs that can readily be used for machine learning
    
    """
    
    data_list = []
    
    for (smiles, y_val) in zip(x_smiles, y):
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)
        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))
        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))
        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
            
        X = torch.tensor(X, dtype = torch.float)
        
        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)
        
        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))
        
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        
        EF = torch.tensor(EF, dtype = torch.float)
        
        # construct label tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)
        
        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))
    return data_list

### Create datasets

In [116]:
pesticides_graph = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(pesticides_logp['no_salt'], pesticides_logp['alogp'])
insecticides_graph = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(insecticides_logp['no_salt'], insecticides_logp['alogp'])

In [117]:
torch.save(pesticides_graph, "pesticides_graphs.pt")
torch.save(insecticides_graph, "insecticides_graphs.pt")